<a href="https://colab.research.google.com/github/Seungwon-hub/ESAA/blob/main/09_12_%E1%84%89%E1%85%A6%E1%84%89%E1%85%A7%E1%86%AB_%E1%84%86%E1%85%A9%E1%84%83%E1%85%A6%E1%86%AF%E1%84%92%E1%85%AE%E1%86%AB%E1%84%85%E1%85%A7%E1%86%AB_%E1%84%8B%E1%85%A7%E1%86%AB%E1%84%89%E1%85%B3%E1%86%B8%E1%84%86%E1%85%AE%E1%86%AB%E1%84%8C%E1%85%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___

확률적 경사 하강법과 미니 배치 경사 하강법을 사용할 수 있다. 전체 훈련 세트를 이용하는 것보다 속도가 더 빠를 것이다. 일반적인 경사 하강법을 사용하면 메모리 공간이 부족할 수 있다.

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

검증 오차가 일정하게 상승하는 현상은 여러 이유로 발생할 수 있다.

첫 번째 이유는 과적합이다. 모델이 훈련 데이터에 너무 과하게 맞춰져서 훈련 데이터에는 성능이 좋지만, 처음 보는 데이터에 대해서는 성능이 떨어질 수 있다. 에포크가 진행될수록 모델이 훈련 데이터의 노이즈와 특이 패턴까지 학습하게 되면서, 일반화 능력이 저하되고 검증 오차가 상승하게 된다. 이는 조기 종료, 정규화, Data augmentation으로 해결할 수 있다.

두 번째 이유는 Learning rate가 너무 높은 경우이다. 학습률이 너무 높으면 오히려 발산할 수 있다. 이는 학습률 스케줄링, 학습률 탐색 등의 방법으로 해결할 수 있다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___



훈련 오차와 검증 오차가 모두 높고 비슷한 값을 나타내는 경우는 모델이 훈련 데이터의 패턴을 제대로 학습하지 못했음을 의미한다. 이런 상황은 높은 편향이 문제다. 그렇기에 모델의 복잡도를 증가시켜야 하고, 규제의 강도를 줄여서 모델이 훈련 데이터를 더 잘 학습하도록 만들어야 한다. 결론적으로 규제 하이퍼파라미터인 알파를 줄여야 한다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

1. 릿지 회귀를 사용해야 하는 이유는 L2 규제항을 추가하여 가중치의 크기를 제한해 높은 다중공선성 문제를 해결하고 모델의 과적합을 방지하여 일반화 성능을 향상시킬 수 있기 때문이다.

2. 릿지 회귀는 모든 특성을 유지하지만 라쏘 회귀는 불필요한 특성을 자동으로 제거하여 모델을 더 간결하고 해석하기 쉽게 만든다.

3. 엘라스틱넷은 L1규제 + L2규제 모델로, 라쏘의 특성 선택 능력과 릿지의 안정성을 모두 가진다. 많은 수의 특성 중에서 일부를 선택하고 동시에 상관관계가 높은 특성들을 안정적으로 처리해야 할 때 사용하는 것이 좋다.

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [2]:
import numpy as np
from sklearn.datasets import load_iris


iris = load_iris()
X = iris.data
y = iris.target

# 데이터 전처리
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# One-hot 인코딩
num_classes = len(np.unique(y))
y_one_hot = np.zeros((y.shape[0], num_classes))
y_one_hot[np.arange(y.shape[0]), y] = 1

# 데이터 분할 (훈련:검증 = 80:20)
def custom_train_test_split(X, y, test_size=0.2, random_state=42):
    np.random.seed(random_state)
    n_samples = X.shape[0]
    n_test = int(n_samples * test_size)
    indices = np.random.permutation(n_samples)
    test_indices = indices[:n_test]
    train_indices = indices[n_test:]
    X_train, X_val = X[train_indices], X[test_indices]
    y_train, y_val = y[train_indices], y[test_indices]
    return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = custom_train_test_split(X, y_one_hot, test_size=0.2)

# 하이퍼파라미터 설정
learning_rate = 0.1
epochs = 500
input_size = X_train.shape[1]
output_size = num_classes

# 초기 가중치 및 편향
W = np.zeros((input_size, output_size))
b = np.zeros(output_size)

# 조기 종료 설정
best_loss = float('inf')
patience = 10
patience_counter = 0

# 2. 소프트맥스 함수
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# 3. 크로스 엔트로피 손실 함수
def cross_entropy_loss(y_true, y_pred):
    m = y_true.shape[0]
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = -np.sum(y_true * np.log(y_pred)) / m
    return loss

# 4. 경사 하강법을 위한 경사(Gradient) 계산
def compute_gradients(X, y_true, y_pred):
    m = X.shape[0]
    error = y_pred - y_true
    dW = X.T @ error / m
    db = np.sum(error, axis=0) / m
    return dW, db

# 5. 훈련 루프
for epoch in range(epochs):
    # 순전파
    z = X_train @ W + b
    y_pred = softmax(z)

    # 손실 계산
    train_loss = cross_entropy_loss(y_train, y_pred)

    # 검증 손실 계산
    z_val = X_val @ W + b
    y_pred_val = softmax(z_val)
    val_loss = cross_entropy_loss(y_val, y_pred_val)

    # 경사 계산 및 가중치 업데이트
    dW, db = compute_gradients(X_train, y_train, y_pred)
    W -= learning_rate * dW
    b -= learning_rate * db

    # 조기 종료 조건 검사
    if val_loss < best_loss:
        best_loss = val_loss
        patience_counter = 0
        best_W = W.copy()
        best_b = b.copy()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"조기 종료! (에포크: {epoch+1}, 검증 손실: {val_loss:.4f})")
            W = best_W
            b = best_b
            break

    if (epoch + 1) % 50 == 0:
        print(f"에포크 {epoch+1}/{epochs}, 훈련 손실: {train_loss:.4f}, 검증 손실: {val_loss:.4f}")

# 6. 최종 모델 성능 평가
def predict(X, W, b):
    z = X @ W + b
    probabilities = softmax(z)
    return np.argmax(probabilities, axis=1)

def accuracy(y_true, y_pred):
    return np.mean(np.argmax(y_true, axis=1) == y_pred)

train_acc = accuracy(y_train, predict(X_train, W, b))
val_acc = accuracy(y_val, predict(X_val, W, b))

print("-" * 30)
print(f"최종 훈련 정확도: {train_acc:.4f}")
print(f"최종 검증 정확도: {val_acc:.4f}")

에포크 50/500, 훈련 손실: 0.4127, 검증 손실: 0.3635
에포크 100/500, 훈련 손실: 0.3407, 검증 손실: 0.2857
에포크 150/500, 훈련 손실: 0.3001, 검증 손실: 0.2454
에포크 200/500, 훈련 손실: 0.2707, 검증 손실: 0.2182
에포크 250/500, 훈련 손실: 0.2478, 검증 손실: 0.1978
에포크 300/500, 훈련 손실: 0.2293, 검증 손실: 0.1818
에포크 350/500, 훈련 손실: 0.2140, 검증 손실: 0.1689
에포크 400/500, 훈련 손실: 0.2011, 검증 손실: 0.1581
에포크 450/500, 훈련 손실: 0.1902, 검증 손실: 0.1491
에포크 500/500, 훈련 손실: 0.1807, 검증 손실: 0.1414
------------------------------
최종 훈련 정확도: 0.9583
최종 검증 정확도: 1.0000
